In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import os

import plotly.express as px
import plotly.graph_objects as go

from scipy.spatial import Voronoi

# Explore data
What is my goal for this project? What steps am I going to take?

Terminology I need to define
* Division
* Booth
* AEC
* 2019 election information: why (booths change each election, diviosn borders change frequently as well)
* PPVC pre polling voting centre

In this document I want to:
* Explore current divisions and polling booths
* Plot them using plotly: have 2019 division as one region, and the booths shown inside. 
* Create a voronoi tesselation for the booths
* Plot the voronoi tesselation


## Get data
Data is available from the Australian Electoral Commision 

In [2]:
booth_info_loc = 'data/20190518/GeneralPollingPlacesDownload-24310.csv'

try:
    booths = pd.read_csv(booth_info_loc,
                         skiprows=1)
except IOError:
    booths = pd.read_csv('https://results.aec.gov.au/24310/Website/Downloads/GeneralPollingPlacesDownload-24310.csv')
    booths.to_csv(booth_info_loc)
    booths = pd.read_csv(booth_info_loc,
                         skiprows=1)

booths.head()

,State,DivisionID,DivisionNm,PollingPlaceID,PollingPlaceTypeID,PollingPlaceNm,PremisesNm,PremisesAddress1,PremisesAddress2,PremisesAddress3,PremisesSuburb,PremisesStateAb,PremisesPostCode,Latitude,Longitude
0,ACT,318,Bean,93925,5,Belconnen BEAN PPVC,Belconnen Community Centre,26 Chandler St,NaN,NaN,BELCONNEN,ACT,2900.0,-35.238930,149.069655
1,ACT,318,Bean,93927,5,BLV Bean PPVC,BLV Canberra,50 Marcus Clarke St,NaN,NaN,CANBERRA CITY,ACT,2601.0,-35.277334,149.126869
2,ACT,318,Bean,11877,1,Bonython,Bonython Primary School,64 Hurtle Ave,NaN,NaN,BONYTHON,ACT,2905.0,-35.431800,149.083000
3,ACT,318,Bean,11452,1,Calwell,Calwell High School,111 Casey Cres,NaN,NaN,CALWELL,ACT,2905.0,-35.440670,149.117600
4,ACT,318,Bean,8761,1,Chapman,Chapman Primary School,46-50 Perry Dr,NaN,NaN,CHAPMAN,ACT,2611.0,-35.356400,149.042000


# Explore with a small subset

I lived in the 2500 postcode for a while, so I am fimiliar with the locationsa so this makes it easier to check that it is working.


In [3]:

booths2500 = booths[booths['PremisesPostCode']==2500]
booths2500

,State,DivisionID,DivisionNm,PollingPlaceID,PollingPlaceTypeID,PollingPlaceNm,PremisesNm,PremisesAddress1,PremisesAddress2,PremisesAddress3,PremisesSuburb,PremisesStateAb,PremisesPostCode,Latitude,Longitude
724,NSW,114,Cunningham,65460,5,BLV Cunningham PPVC,BLV Cunningham,Corporate Square,Level 4,43 Burelli St,WOLLONGONG,NSW,2500.0,-34.427135,150.898067
729,NSW,114,Cunningham,539,1,Coniston,Coniston Public School,123 Auburn St,NaN,NaN,CONISTON,NSW,2500.0,-34.438600,150.887000
735,NSW,114,Cunningham,30105,5,Divisional Office (PREPOLL),Divisional Office,Corporate Square,Level 4,43 Burelli St,WOLLONGONG,NSW,2500.0,-34.427011,150.898107
739,NSW,114,Cunningham,544,1,Gwynneville,Gwynneville Primary School,10A Acacia Ave,NaN,NaN,GWYNNEVILLE,NSW,2500.0,-34.418079,150.879339
742,NSW,114,Cunningham,546,1,Keiraville,Keiraville Public School,286 Gipps Rd,NaN,NaN,KEIRAVILLE,NSW,2500.0,-34.414700,150.873000
744,NSW,114,Cunningham,555,1,Mangerton,Mount St Thomas Public School,12-14 Taronga Ave,NaN,NaN,MANGERTON,NSW,2500.0,-34.437281,150.870466
745,NSW,114,Cunningham,553,1,Mount Keira,Edmund Rice College,112 Mount Keira Rd,NaN,NaN,WEST WOLLONGONG,NSW,2500.0,-34.419980,150.863017
765,NSW,114,Cunningham,565,1,Wollongong,Wollongong Town Hall,93 Crown St,NaN,NaN,WOLLONGONG,NSW,2500.0,-34.425718,150.897710
766,NSW,114,Cunningham,34033,5,Wollongong CUNNINGHAM PPVC,3/51 Crown St,NaN,NaN,NaN,WOLLONGONG,NSW,2500.0,-34.426057,150.899519
767,NSW,114,Cunningham,83535,1,Wollongong East,Wollongong Public School,67a Church St,NaN,NaN,WOLLONGONG,NSW,2500.0,-34.422548,150.896217


In [4]:
fig = px.scatter_mapbox(booths2500, lat="Latitude", lon="Longitude", hover_name="PremisesNm", 
                        hover_data=["DivisionNm", "PollingPlaceNm"],
                        zoom=11, height=300)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

First problem I can see: what is a booth doing in Nowra? That is definitely not in the 2500 postcode. The address for this booth is actually in central Wollongong and there are 5 others with the same address, but all with different coordinates. 

In [5]:
booths2500[booths2500['PremisesAddress3']=='43 Burelli St']

,State,DivisionID,DivisionNm,PollingPlaceID,PollingPlaceTypeID,PollingPlaceNm,PremisesNm,PremisesAddress1,PremisesAddress2,PremisesAddress3,PremisesSuburb,PremisesStateAb,PremisesPostCode,Latitude,Longitude
724,NSW,114,Cunningham,65460,5,BLV Cunningham PPVC,BLV Cunningham,Corporate Square,Level 4,43 Burelli St,WOLLONGONG,NSW,2500.0,-34.427135,150.898067
735,NSW,114,Cunningham,30105,5,Divisional Office (PREPOLL),Divisional Office,Corporate Square,Level 4,43 Burelli St,WOLLONGONG,NSW,2500.0,-34.427011,150.898107
1069,NSW,120,Gilmore,65612,5,BLV Gilmore PPVC,BLV Gilmore,Corporate Square,Level 4,43 Burelli St,WOLLONGONG,NSW,2500.0,-34.875295,150.603992
1079,NSW,120,Gilmore,30111,5,Divisional Office (PREPOLL),Divisional Office,Corporate Square,Level 4,43 Burelli St,WOLLONGONG,NSW,2500.0,-34.427011,150.898107
3029,NSW,150,Whitlam,65458,5,BLV Whitlam PPVC,BVL Whitlam,Corporate Square,Level 4,43 Burelli St,WOLLONGONG,NSW,2500.0,-34.427135,150.898067
3042,NSW,150,Whitlam,30141,5,Divisional Office (PREPOLL),Divisional Office,Corporate Square,Level 4,43 Burelli St,WOLLONGONG,NSW,2500.0,-34.427135,150.898067


These are all `PollingPlaceTypeID` = 5. I suspect that these might be for prepolling or some sort of admin that is based out of the Burelli street office, but represents different real locations. Before continuing, I want to check what `PollingPlaceTypeID` represents:

In [6]:
booths.groupby('PollingPlaceTypeID')['DivisionID'].count()

PollingPlaceTypeID
1    7169
2     489
3      42
4      26
5    1149
Name: DivisionID, dtype: int64

In [7]:
booths.groupby('PollingPlaceTypeID').head(5).sort_values('PollingPlaceTypeID')

,State,DivisionID,DivisionNm,PollingPlaceID,PollingPlaceTypeID,PollingPlaceNm,PremisesNm,PremisesAddress1,PremisesAddress2,PremisesAddress3,PremisesSuburb,PremisesStateAb,PremisesPostCode,Latitude,Longitude
2,ACT,318,Bean,11877,1,Bonython,Bonython Primary School,64 Hurtle Ave,NaN,NaN,BONYTHON,ACT,2905.0,-35.431800,149.083000
3,ACT,318,Bean,11452,1,Calwell,Calwell High School,111 Casey Cres,NaN,NaN,CALWELL,ACT,2905.0,-35.440670,149.117600
4,ACT,318,Bean,8761,1,Chapman,Chapman Primary School,46-50 Perry Dr,NaN,NaN,CHAPMAN,ACT,2611.0,-35.356400,149.042000
5,ACT,318,Bean,8763,1,Chisholm,Caroline Chisholm School,108 Hambidge Cres,NaN,NaN,CHISHOLM,ACT,2905.0,-35.419522,149.122539
6,ACT,318,Bean,93916,1,City (Bean),Pilgrim House,69 Northbourne Ave,NaN,NaN,CANBERRA CITY,ACT,2601.0,-35.276702,149.129081
40,ACT,318,Bean,93923,2,Special Hospital Team 2,Multiple sites,NaN,NaN,NaN,NaN,ACT,NaN,NaN,NaN
39,ACT,318,Bean,93921,2,Special Hospital Team 1,Multiple sites,NaN,NaN,NaN,NaN,ACT,NaN,NaN,NaN
41,ACT,318,Bean,93924,2,Special Hospital Team 3,Multiple sites,NaN,NaN,NaN,NaN,ACT,2611.0,NaN,NaN
85,ACT,101,Canberra,32712,2,Special Hospital Team 1,Multiple sites,NaN,NaN,NaN,NaN,ACT,NaN,NaN,NaN
86,ACT,101,Canberra,58810,2,Special Hospital Team 2,Multiple sites,NaN,NaN,NaN,NaN,ACT,NaN,NaN,NaN


It appears that the values for `PollingPlaceTypeID`
* 1: a normal on the day polling booth
* 2: a mobile team that visits hospitals
* 3: a mobile team that visits remote locations
* 4: other mobile team 
* 5: prepolling voting centre? 

For now I'll exclude everything other than 1. My goal right now is a proof of concept and not to overcomplicate things. I'll need to come back and double check the remote locations as it could be quite easy to have large parts of Australia underrepresented. 


In [5]:
booths_normal = booths[booths['PollingPlaceTypeID']==1]
booths2500 = booths_normal[booths_normal['PremisesPostCode']==2500]

In [9]:
fig = px.scatter_mapbox(booths2500, lat="Latitude", lon="Longitude", hover_name="PremisesNm", 
                        hover_data=["DivisionNm", "PollingPlaceNm"],
                        zoom=11, height=300)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Plotting the normal booths, everything seems fine. Next lets make the voronoi tesselation.

# Making a voronoi tesselation
I want to define a region around each booth to capture the area it represents. This is a simplification as it assumes that voters travel to their nearest booth ignoring population distribution and the physical georgraphy. More could be done with ABS data, but for now this will be fine.

SciPy has a package that generates the voronoi tesselation. It returns
* `vertices`: the finite vertices of the voronoi regions
* `regions`: each array is a region. The region is defined by the index of `vertices` that make up the region. if the index is `-1` that region goes to infinity
* `ridge_vertices`: defines edges of the voronoi regions. Once again defined by the indices of `vertices`
* `ridge_points`: the two input points (defined by the index of the input points) that corresponds to the ridge vertex at the same index. 
* `point_region`: maps the voronoi region index to its corresponding input point




In [24]:
vor_df = booths2500.copy()
vor_df['id'] = vor_df.reset_index().index.values

vor = Voronoi(vor_df[['Longitude', 'Latitude']].values)

To plot the regions I'm using the __[choropleth graph](https://plotly.com/python/mapbox-county-choropleth/)__. I need to get the voronoi region in a json format and any other information I want in the hover over text in a dataframe. The ID used to join the object is the index

In [74]:

# Map the region index to the input index
region_point = {region_index:input_index for input_index, region_index in enumerate(vor.point_region)}

json_voronoi = {"type": "FeatureCollection",
                "features": [{'type':'Feature',
                              'id': region_point[r],
                              'geometry':{"type": "Polygon",
                                          "coordinates":[[vor.vertices[point] for point in region if point != -1]]
                                         }
                             }
                             for r, region in enumerate(vor.regions) if len(region) > 0]
                }



In [195]:
fig = px.choropleth_mapbox(vor_df,
                           geojson=json_voronoi,
                           locations='id',
                           color='id',
                           color_continuous_scale="Viridis",
                           range_color=(0, 10),
                           mapbox_style="carto-positron",
                           zoom=11,
                           hover_data=['PremisesNm', 'Longitude', 'Latitude'],
                           opacity=0.5,
                           labels={'id':'Input ID'},
                           center={"lat": -34.4264059, "lon": 150.893344},
                          )

fig.add_trace(
px.scatter_mapbox(vor_df, lat="Latitude", lon="Longitude", hover_name="PremisesNm", 
                  hover_data=["DivisionNm", "PollingPlaceNm", 'PollingPlaceID'],
                  zoom=11, height=300).data[0]
)


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


KeyboardInterrupt: 

Not all the input points have a region around them. This is because these regions go out to infinity. To define them we need to define what is meant by infinity for these points and then do some maths to define the remaining ridge vertices. 

* Each edge of a voronoi polygon is called a *ridge*
* Each ridge is perpendicular to the line drawn between the centre of two adjacent regions, called the *tangent*
* `vor.ridge_points` contains the index of the input points that should have a ridge between them (i.e. the regions that are adjacent to each other)
* The tangent vector is the normalised difference between the two centres
* The normal of a 2D vector, (v1, v2) is (-v2, v1): this is the direction vector for the new ridge line
* We want the ridge line to be point away from the center of the entire voronoi region. We take the difference of the midpoint of the two centres (a point that must be on the new ridge) and the centre of all input points which gives us a vector that should either point towards the centre or to infinity. We take the dot product with the normal which gives the cosine of the angle beween the two vectors. The sign of the cosine is then applied to the normal to make sure it is moving towards infinity, rather than the centre
* The next vertex point is calculated as the already known vertex point (one is always known as it is the border of a region) plus the direction vector multiplied by a radius.

In [134]:
def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions. Taken from https://stackoverflow.com/questions/57385472/how-to-set-a-fixed-outer-boundary-to-voronoi-tessellations
    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.
    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()
    new_point_region = []

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]
        
        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            tangent = vor.points[p2] - vor.points[p1] # tangent
            tangent = tangent / np.linalg.norm(tangent)
            normal = np.array([-tangent[1], tangent[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, normal)) * normal
            far_point = (vor.vertices[v2] + direction * radius)

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)




vor = Voronoi(booths2500[['Longitude', 'Latitude']].values)

regions, vertices = voronoi_finite_polygons_2d(vor, .05)

# region_point is now not needed as the region index maps to the input point index
json_voronoi = {"type": "FeatureCollection",
                "features": [{'type':'Feature',
                            'id': r,
                            'geometry':{"type": "Polygon",
                                        "coordinates":[[vertices[point] for point in region]]
                                        }
                            }
                            for r, region in enumerate(regions) if len(region) > 0]
            }



In [135]:
fig = px.choropleth_mapbox(vor_df,
                           geojson=json_voronoi,
                           locations='id',
                           color='id',
                           color_continuous_scale="Viridis",
                           range_color=(0, 10),
                           mapbox_style="carto-positron",
                           zoom=11,
                           hover_data=['PremisesNm', 'Longitude', 'Latitude'],
                           opacity=0.5,
                           labels={'id':'Input ID'},
                           center={"lat": -34.4264059, "lon": 150.893344},
                          )

fig.add_trace(
px.scatter_mapbox(vor_df, lat="Latitude", lon="Longitude", hover_name="PremisesNm", 
                  hover_data=["DivisionNm", "PollingPlaceNm", 'PollingPlaceID'],
                  zoom=11, height=300).data[0]
)


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


The next step is to test it for all of NSW


In [6]:
def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions. Taken from https://stackoverflow.com/questions/57385472/how-to-set-a-fixed-outer-boundary-to-voronoi-tessellations
    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.
    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()
    new_point_region = []

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]
        
        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            tangent = vor.points[p2] - vor.points[p1] # tangent
            tangent = tangent / np.linalg.norm(tangent)
            normal = np.array([-tangent[1], tangent[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, normal)) * normal
            far_point = (vor.vertices[v2] + direction * radius)

            # if far point is wrong, print all the info
            if far_point[0] < 80:
                print('p1, p2', p1, p2)
                print('v1, v2', v1, v2)
                print('tangent', tangent)
                print('normal', normal)
                print('midpoint', midpoint)
                print('direction', direction)
                print('far_point', far_point)
                print('\n\n\n')

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)


booths_nsw = booths_normal[booths_normal['State']=='NSW']

booths_nsw['id'] = booths_nsw.reset_index().index.values

vor = Voronoi(booths_nsw[['Longitude', 'Latitude']].values)

# regions, vertices = voronoi_finite_polygons_2d(vor, .005)

  

# region_point is now not needed as the region index maps to the input point index
json_voronoi = {"type": "FeatureCollection",
                "features": [{'type':'Feature',
                            'id': r,
                            'geometry':{"type": "Polygon",
                                        "coordinates":[[vertices[point] for point in region]]
                                        }
                            }
                            for r, region in enumerate(regions) if len(region) > 0]
                }


fig = px.choropleth_mapbox(booths_nsw,
                           geojson=json_voronoi,
                           locations='id',
                           color='id',
                           color_continuous_scale="Viridis",
                           range_color=(0, 2500),
                           mapbox_style="carto-positron",
                           hover_data=['PremisesNm', 'Longitude', 'Latitude'],
                           opacity=0.5,
                           labels={'id':'Input ID'},
                           center={"lat": -32, "lon": 146},
                           zoom=3,
                          )

fig.add_trace(
px.scatter_mapbox(booths_nsw, lat="Latitude", lon="Longitude", hover_name="PremisesNm", 
                  hover_data=["DivisionNm", "PollingPlaceNm", 'PollingPlaceID'], height=300).data[0]
)

fig.add_trace(
px.scatter_mapbox(bad, lat="Latitude", lon="Longitude", hover_name="PremisesNm", 
                  hover_data=["DivisionNm", "PollingPlaceNm", 'PollingPlaceID'], height=300, color_continuous_scale=px.colors.cyclical.IceFire, color='id').data[0]
)


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()



NameError: name 'regions' is not defined

Problems here
* Lord Howe islandisn't being included?
* Things out west is pointing in the wrong direction


In [179]:
[v for v, vertex in enumerate(vor.vertices) if vertex[0] < 140]

[20, 21, 76, 134]

In [180]:
vor.vertices[76]

array([ -95.98305617, 1740.03180803])

In [181]:
vor.vertices[[20, 21, 76, 134]]

array([[ 139.58919591,  -39.01158422],
       [ 139.97488367,  -38.46274621],
       [ -95.98305617, 1740.03180803],
       [ 136.96530577,  -14.65022248]])

In [192]:
-95.98305617+180

84.01694383

In [194]:
1740.03180803 -180*9

120.0318080300001

In [172]:
# What regions is this point a part of 76

[r for r, region in enumerate(vor.regions) if 76 in region]

[24, 64, 93]

In [175]:
# These centres are
[point_index for point_index, region_index in enumerate(vor.point_region) if region_index==24 or region_index==64 or region_index==93]

[1733, 1735, 1987]

In [182]:
vor.points[[1733, 1735, 1987]]

array([[150.363    , -28.6153   ],
       [149.577    , -28.7246   ],
       [153.5421435, -28.1695795]])

In [188]:
bad = booths_nsw[booths_nsw['id'].isin([1733, 1735, 1987])] 
bad 


In [150]:
# Find the vertices that are well beyond Australia
[v for v, vertex in enumerate(vertices) if vertex[0] < 80]

[76, 4569, 4576]

In [151]:
# Coordinates of these vertices
vertices[[76, 4569, 4576]]

array([[ -95.98305617, 1740.03180803],
       [ -95.98374929, 1740.03675976],
       [ -95.98374929, 1740.03675976]])

These are completely out. I suspect what is happening is when calculating the location, the centroid is so far from the overall center that we end up having overflow errors. I'm going to go back and check what is happening at those points. In the end the problem is probably something to do with the voronoi algorithm not working with spherical coordinates. To avoid this we can first project to a mercator projection


In [113]:
booths_nsw[booths_nsw['PollingPlaceID']==2755]

,State,DivisionID,DivisionNm,PollingPlaceID,PollingPlaceTypeID,PollingPlaceNm,PremisesNm,PremisesAddress1,PremisesAddress2,PremisesAddress3,PremisesSuburb,PremisesStateAb,PremisesPostCode,Latitude,Longitude,id
2770,NSW,149,Sydney,2755,1,Lord Howe Island,Lord Howe Island Central School,Lagoon Rd,NaN,NaN,LORD HOWE ISLAND,NSW,2898.0,-31.529895,159.06916,2192


In [115]:
# Lord Howe Island region: all the vertices in 
regions[2192]

[42, 14, 2, 3, 1, 4578, 4579, 4, 470, 471, 312, 328, 800, 801, 353]

[76, 4569, 4576]

array([[ -95.98305617, 1740.03180803],
       [ -95.9831948 , 1740.03279838],
       [ -95.9831948 , 1740.03279838]])

In [122]:
# What region are these used for
[r for r, region in enumerate(regions) if 76 in region]

[1733, 1735, 1987]

In [123]:
booths_nsw.iloc[[1733, 1735, 1987]]
# It is really strange that Parkes 

,State,DivisionID,DivisionNm,PollingPlaceID,PollingPlaceTypeID,PollingPlaceNm,PremisesNm,PremisesAddress1,PremisesAddress2,PremisesAddress3,PremisesSuburb,PremisesStateAb,PremisesPostCode,Latitude,Longitude,id
2208,NSW,139,Parkes,1072,1,Boggabilla,Boggabilla Central School,16 South St,NaN,NaN,BOGGABILLA,NSW,2409.0,-28.615300,150.363000,1733
2210,NSW,139,Parkes,1074,1,Boomi,Boomi Public School,25 Werrina St,NaN,NaN,BOOMI,NSW,2405.0,-28.724600,149.577000,1735
2521,NSW,145,Richmond,2472,1,Tweed Heads,Tweed Heads Public School,1-5 Stuart St,NaN,NaN,TWEED HEADS,NSW,2485.0,-28.169579,153.542144,1987


In [36]:
import geopandas as gpd

booths_nsw = gpd.GeoDataFrame(booths_normal[booths_normal['State']=='NSW'])
booths_nsw['geometry'] = gpd.points_from_xy(booths_nsw['Longitude'], booths_nsw['Latitude'], crs='EPSG:4326')
booths_nsw = booths_nsw.to_crs('EPSG:3395')
booths_nsw['id'] = booths_nsw.reset_index().index.values

coords = [(c.x, c.y) for c in booths_nsw['geometry']]

vor = Voronoi(coords)
regions, vertices = voronoi_finite_polygons_2d(vor, .1)
#booths_nsw.head()

p1, p2 1735 1987
v1, v2 -1 45
tangent [0.98769016 0.15642301]
normal [-0.15642301  0.98769016]
midpoint [16871534.35205661 -3285156.53093445]
direction [-0.15642301  0.98769016]
far_point [-2.41135589e+07  2.55503972e+08]




p1, p2 1987 1735
v1, v2 -1 45
tangent [-0.98769016 -0.15642301]
normal [ 0.15642301 -0.98769016]
midpoint [16871534.35205661 -3285156.53093445]
direction [-0.15642301  0.98769016]
far_point [-2.41135589e+07  2.55503972e+08]






In [37]:
vertices_t = gpd.GeoSeries(gpd.points_from_xy([v[0] for v in vertices], [v[1] for v in vertices], crs='EPSG:3395'))
vertices_t = vertices_t.to_crs('EPSG:4326')
vertices = [(v.x, v.y)for v in vertices_t.values]

In [38]:


# region_point is now not needed as the region index maps to the input point index
# Map the region index to the input index


json_voronoi = {"type": "FeatureCollection",
                "features": [{'type':'Feature',
                            'id': r,
                            'geometry':{"type": "Polygon",
                                        "coordinates":[[vertices[point] for point in region]]
                                        }
                            }
                            for r, region in enumerate(regions) if len(region) > 0]
                }


fig = px.choropleth_mapbox(booths_nsw,
                           geojson=json_voronoi,
                           locations='id',
                           color='id',
                           color_continuous_scale="Viridis",
                           range_color=(0, 2500),
                           mapbox_style="carto-positron",
                           hover_data=['PremisesNm', 'Longitude', 'Latitude'],
                           opacity=0.5,
                           labels={'id':'Input ID'},
                           center={"lat": -32, "lon": 146},
                           zoom=3,
                          )

fig.add_trace(
px.scatter_mapbox(booths_nsw, lat="Latitude", lon="Longitude", hover_name="PremisesNm", 
                  hover_data=["DivisionNm", "PollingPlaceNm", 'PollingPlaceID'], height=300).data[0]
)

# fig.add_trace(
# px.scatter_mapbox(bad, lat="Latitude", lon="Longitude", hover_name="PremisesNm", 
#                   hover_data=["DivisionNm", "PollingPlaceNm", 'PollingPlaceID'], height=300, color_continuous_scale=px.colors.cyclical.IceFire, color='id').data[0]
# )


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [50]:
from shapely.geometry import Polygon
# Get booth data
# booths_nsw = gpd.GeoDataFrame(booths_normal[booths_normal['State']=='NSW'])
# booths_nsw['id'] = booths_nsw.reset_index().index.values

# # Convert to other coordinates
# booths_nsw['geometry'] = gpd.points_from_xy(booths_nsw['Longitude'], booths_nsw['Latitude'], crs='EPSG:4326')
# booths_nsw = booths_nsw.to_crs('EPSG:3395')

# coords = [(c.x, c.y) for c in booths_nsw['geometry']]

# # Voronoi
# vor = Voronoi(coords)
# regions, vertices = voronoi_finite_polygons_2d(vor, .1)

# # Convert back to normal systems
# vertices_t = gpd.GeoSeries(gpd.points_from_xy([v[0] for v in vertices], [v[1] for v in vertices], crs='EPSG:3395'))
# vertices_t = vertices_t.to_crs('EPSG:4326')
# vertices = [(v.x, v.y)for v in vertices_t.values]

# trim the vertices so that they don't go past state border
for region in regions:
    # Make region polygon
    polygon = Polygon([vertices[v] for v in region])

    # Intersection with state
    polygon = polygon.intersection(nsw_border_p)

    # Convert back to coordinates

TopologyException: Input geom 0 is invalid: Self-intersection at or near point 143.384215091725 89.999998161716633 at 143.384215091725 89.999998161716633


TopologicalError: The operation 'GEOSIntersection_r' could not be performed. Likely cause is invalidity of the geometry <shapely.geometry.polygon.Polygon object at 0x7fc0b1306c40>

In [56]:
list(polygon.interiors)

[]

In [51]:
from shapely.ops import cascaded_union

# nsw_border = gpd.read_file('data/State Boundaries AUGUST 2020 - Copy/Standard/NSW_STATE_POLYGON_shp.shp')
nsw_border_p = cascaded_union(nsw_border['geometry'])
nsw_border_p

KeyboardInterrupt: 

In [53]:
nsw_border_p.is_valid

True